<a href="https://colab.research.google.com/github/kdemertzis/LNexamples/blob/main/drones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install torch==1.8.0 torchvision==0.9.0 opencv-python==4.5.1.48 numpy==1.19.3 scipy==1.5.4

# Import required libraries
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.autograd import Variable
from torchvision import transforms
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords
from utils.datasets import letterbox
from sort import Sort
from google.colab import files
import urllib.request

# Download the pre-trained YOLOv5 model
url = 'https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt'
filename = 'yolov5s.pt'
urllib.request.urlretrieve(url, filename)

# Load the pre-trained YOLOv5 model
model = attempt_load(filename, map_location=torch.device('cpu'))
model.conf = 0.5
model.eval()

# Define the classes
classes = ['non-alert', 'alert']

# Define the transforms for the input image
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Initialize the SORT tracker
tracker = Sort()

# Upload the video file
uploaded = files.upload()

# Open the video file
video_name = next(iter(uploaded))
cap = cv2.VideoCapture(video_name)

# Loop for real-time prediction on each frame of the video
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        # End of video
        break

    # Preprocess the image
    img = transform(frame)
    img = img.unsqueeze(0)

    # Make prediction
    with torch.no_grad():
        output = model(img)
        output = non_max_suppression(output, conf_thres=0.5, iou_thres=0.5, classes=None, agnostic=False)

    # Initialize the tracked objects
    tracked_objects = []

    # Draw the labels and boxes on the image and track the objects
    if output[0] is not None:
        for det in output[0]:
            xyxy = scale_coords(img.shape[2:], det[:4], frame.shape).round()
            label = int(det[5])
            cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
            cv2.putText(frame, classes[label], (xyxy[0], xyxy[1]), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
            tracked_objects.append([xyxy[0], xyxy[1], xyxy[2], xyxy[3], label])

    # Update the tracker
    if len(tracked_objects) > 0:
        tracked_objects = np.array(tracked_objects)
        tracked_objects = tracker.update(tracked_objects)

        # Draw the boxes and IDs of the tracked objects
        for obj in tracked_objects:
            x1, y1, x2, y2, obj_id = obj.astype(np.int32, (0, 0, 255), 2)
        # Check if any tracked object is in alert class
        alert_objects = [obj for obj in tracked_objects if obj[4] == 1]
        if len(alert_objects) > 0:
        # Trigger alert
           print('ALERT: %d objects detected in the alert class' % len(alert_objects))

        # Display the resulting frame
        cv2.imshow('frame', frame)

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
           break

cap.release()
cv2.destroyAllWindows()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 744.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0
    Uninstalling torch-1.9.0:
      Successfully uninstalled torch-1.9.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: opencv-python
    Found ex

ModuleNotFoundError: ignored